In [1]:
from deap import base, creator, tools, algorithms
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as lines
import random
from mpl_toolkits.basemap import Basemap

print "PHY905 Project 3: Traveling Salesman"
print "by Steve Hughey"
print "5/4/15"

# Read cities file
cities = [line.strip() for line in open('Cities.txt')]
Ncities = len(cities)
LatLon = [line.strip().strip('{}') for line in open('LatLon.txt')]
city_coords = np.zeros((Ncities,2))
print "Number of cities: %d" % Ncities

for i in range(Ncities):
    tmp = LatLon[i].split(',')
    city_coords[i,0] = np.float(tmp[0])
    city_coords[i,1] = np.float(tmp[1])
    print "[%-2d] %-30s\t(%f,%f)" % (i,cities[i],city_coords[i,0],city_coords[i,1])

# Read distance matrix and store in distMat
# All distances in distMat in units of km
distMat = np.zeros((Ncities,Ncities))
# pLinks: all possible links between cities
pLinks = np.zeros((Ncities*(Ncities-1)/2,2))
dst = [line.strip().split('\t') for line in open('distMatrix.mat')]
cnt = 0
for i in range(Ncities):
    for j in range(i+1,Ncities):
        distMat[i,j] = np.float(dst[i][j-i-1])/1000.0
        distMat[j,i] = np.float(dst[i][j-i-1])/1000.0
        pLinks[cnt,0] = i
        pLinks[cnt,1] = j
        cnt += 1
print pLinks

PHY905 Project 3: Traveling Salesman
by Steve Hughey
5/4/15
Number of cities: 40
[0 ] New York City, New York       	(40.664274,-73.938500)
[1 ] Los Angeles, California       	(34.019394,-118.410825)
[2 ] Chicago, Illinois             	(41.837551,-87.681844)
[3 ] Houston, Texas                	(29.780472,-95.386342)
[4 ] Philadelphia, Pennsylvania    	(40.009375,-75.133346)
[5 ] Phoenix, Arizona              	(33.572162,-112.087966)
[6 ] San Antonio, Texas            	(29.472403,-98.525142)
[7 ] San Diego, California         	(32.815300,-117.134993)
[8 ] Dallas, Texas                 	(32.794176,-96.765503)
[9 ] San Jose, California          	(37.296867,-121.819306)
[10] Indianapolis, Indiana         	(39.777995,-86.145838)
[11] Jacksonville, Florida         	(30.337019,-81.661302)
[12] San Francisco, California     	(37.759881,-122.437392)
[13] Austin, Texas                 	(30.307182,-97.755996)
[14] Columbus, Ohio                	(39.984799,-82.985044)
[15] Fort Worth, Texas       

In [2]:
# Get the distance of a complete circuit around the cities
# Goal is to minimize this function over all possible paths
def pathDistance(path):
    dist = 0.0
    for i in range(Ncities):
        tmp = pLinks[path[i],:]
        dist += distMat[tmp[0],tmp[1]]
    return dist
# Pack and unpack the path:
# unpack takes the global edge list and unpacks it into
# a sequence of cities visited
# pack takes a sequence of cities visited and returns edge IDs
def unpack_path(indiv):
    lst = []
    a = pLinks[indiv[Ncities-1],:]
    b = pLinks[indiv[0],:]
    nxt = np.intersect1d(a,b)
    lst.append(np.int(nxt))
    for i in range(Ncities-1):
        a = pLinks[indiv[i],:]
        b = pLinks[indiv[i+1],:]
        nxt = np.intersect1d(a,b)
        lst.append(np.int(nxt))
    return lst
def pack_path(tmp):
    pth = []
    for i in range(Ncities-1):
        ed = np.sort([tmp[i],tmp[i+1]])
        idx = np.where(np.all(pLinks==ed,axis=1))
        pth.append(int(idx[0]))
    ed = np.sort([tmp[Ncities-1],tmp[0]])
    idx = np.where(np.all(pLinks==ed,axis=1))
    pth.append(int(idx[0]))
    return pth
# Generate a random set of edges that visits each city once
def randomPath():
    tmp = np.random.permutation(Ncities)
    pth = pack_path(tmp)
    return pth
def xover(ind1,ind2):
    uind1 = unpack_path(ind1)
    uind2 = unpack_path(ind2)
    tools.cxOrdered(uind1,uind2)
    cxind1 = creator.Individual(pack_path(uind1))
    cxind2 = creator.Individual(pack_path(uind2))
    return (cxind1,cxind2)
# Mutate a path generated by randomPath or something like it
# with probability mutprb
def mutate(indiv,mutprb=0.15):
    lst = unpack_path(indiv)
    tmp = tools.mutShuffleIndexes(lst,mutprb)[0]
    pth = creator.Individual(pack_path(tmp))
    return (pth,)
#pth = randomPath()
#print pth
#print pack_path(unpack_path(pth))
#print pathDistance(pth)
#print pathDistance(mutate(pth,0.15))

In [12]:
# Create the toolbox for the algorithm
print "Creating DEAP structures..."
toolbox = base.Toolbox()
creator.create("FitnessMin",base.Fitness,weights=(-1.0,))
creator.create("Individual",list,fitness=creator.FitnessMin)

# the register method creates an alias for some function with arguments
# and stores it in the DEAP toolbox
toolbox.register("indices",randomPath)
toolbox.register("individual",tools.initIterate,creator.Individual,toolbox.indices)
toolbox.register("population",tools.initRepeat,list,toolbox.individual)
toolbox.register("mate",xover)
toolbox.register("mutate",mutate,mutprb=0.05)

# Define the fitness function (distance) using the toolbox.individual
# object, output must be iterable tuple
def fitnessFunction(individual):
    return (pathDistance(individual),)

toolbox.register("evaluate",fitnessFunction)
toolbox.register("select",tools.selTournament,tournsize=3)
print "Done"

Creating DEAP structures...
Done


In [13]:
# Generate an initial population
popSize = 200
pop = toolbox.population(n=popSize)
result, log = algorithms.eaSimple(pop,toolbox,cxpb=0.8,mutpb=0.2,ngen=1000,verbose=True)
best_individual = tools.selBest(result, k=1)[0]
print "Fitness of the best individual: ", fitnessFunction(best_individual)
best_individual = unpack_path(best_individual)

gen	nevals
0  	200   
1  	167   
2  	175   
3  	171   
4  	161   
5  	173   
6  	158   
7  	174   
8  	170   
9  	161   
10 	164   
11 	168   
12 	163   
13 	174   
14 	173   
15 	175   
16 	163   
17 	156   
18 	165   
19 	175   
20 	155   
21 	164   
22 	164   
23 	163   
24 	171   
25 	168   
26 	175   
27 	167   
28 	155   
29 	169   
30 	173   
31 	166   
32 	166   
33 	163   
34 	164   
35 	167   
36 	174   
37 	164   
38 	168   
39 	175   
40 	161   
41 	177   
42 	179   
43 	179   
44 	152   
45 	167   
46 	159   
47 	168   
48 	163   
49 	166   
50 	166   
51 	161   
52 	165   
53 	172   
54 	158   
55 	171   
56 	163   
57 	165   
58 	165   
59 	175   
60 	183   
61 	178   
62 	175   
63 	172   
64 	172   
65 	179   
66 	169   
67 	170   
68 	165   
69 	174   
70 	178   
71 	166   
72 	174   
73 	171   
74 	164   
75 	166   
76 	174   
77 	175   
78 	169   
79 	159   
80 	177   
81 	170   
82 	167   
83 	158   
84 	159   
85 	170   
86 	160   
87 	172   
88 	171   
89 	166   

In [18]:
# Draw the map
print [(i+1,cities[best_individual[i]]) for i in range(Ncities)]
lats = [city_coords[best_individual[i],0] for i in range(Ncities)]
lons = [city_coords[best_individual[i],1] for i in range(Ncities)]
lats.append(city_coords[best_individual[0],0])
lons.append(city_coords[best_individual[0],1])
m = Basemap(width=5500000,height=3000000,projection='lcc',
            resolution='l',lat_1=45.,lat_2=45.,lat_0=39,lon_0=-95.)
m.readshapefile('in101503','in101503')
m.drawcoastlines()
m.drawcountries()
m.drawstates()
m.drawmapboundary(fill_color='aqua')
m.fillcontinents(color='gray',lake_color='aqua')
x, y = m(lons,lats)
m.plot(x,y, '-o', markersize=10, linewidth=1, color='k', markerfacecolor='g')
bbox_props = dict(boxstyle="round",fc="w",ec="0.5",alpha=0.5)
for i in range(Ncities):
    icity = best_individual[i]
    a = plt.text(x[i],y[i],cities[icity], ha='center',\
      va='center',bbox=bbox_props,fontsize=8)
    plt.arrow(x[i],y[i],x[i+1]-x[i],y[i+1]-y[i],width=1,head_width=50000,color='b')
plt.show()

[(1, 'Fresno, California'), (2, 'Los Angeles, California'), (3, 'Long Beach, California'), (4, 'San Diego, California'), (5, 'Las Vegas, Nevada'), (6, 'Phoenix, Arizona'), (7, 'Mesa, Arizona'), (8, 'Tucson, Arizona'), (9, 'Albuquerque, New Mexico'), (10, 'El Paso, Texas'), (11, 'San Antonio, Texas'), (12, 'Austin, Texas'), (13, 'Houston, Texas'), (14, 'Dallas, Texas'), (15, 'Fort Worth, Texas'), (16, 'Oklahoma City, Oklahoma'), (17, 'Indianapolis, Indiana'), (18, 'Milwaukee, Wisconsin'), (19, 'Chicago, Illinois'), (20, 'Detroit, Michigan'), (21, 'Columbus, Ohio'), (22, 'Washington, District of Columbia'), (23, 'Baltimore, Maryland'), (24, 'Philadelphia, Pennsylvania'), (25, 'Boston, Massachusetts'), (26, 'New York City, New York'), (27, 'Virginia Beach, Virginia'), (28, 'Charlotte, North Carolina'), (29, 'Jacksonville, Florida'), (30, 'Atlanta, Georgia'), (31, 'Nashville, Tennessee'), (32, 'Memphis, Tennessee'), (33, 'Kansas City, Missouri'), (34, 'Colorado Springs, Colorado'), (35, 'D